In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from rfpimp import *
from pandas.api.types import is_object_dtype,is_string_dtype
import category_encoders as ce

In [21]:
# GENERAL RF MODEL
#def evaluate(X, y): /*/     rf = RandomForestRegressor(n_estimators=100, n_jobs=-1, oob_score=True) /*/     rf.fit(X, y) /*/     oob = rf.oob_score_ /*/    n = rfnnodes(rf) /*/     h = np.median(rfmaxdepths(rf)) /*/     print(f"OOB R^2 is {oob:.5f} using {n:,d} tree nodes with {h} median tree depth") /*/     return rf, oob

In [20]:
#validate model
#X = rent_clean[['bathrooms', 'bedrooms', 'longitude', 'latitude'] /*/ y = rent_clean['price'] /*/ rf, oob = evaluate(X, y)

In [23]:
#FEATURE IMPORTANCE
#def showimp(rf, X, y): /*/     features = list(X.columns) /*/     I = importances(rf, X, y, features=features) /*/     plot_importances(I, color='#4575b4')

In [7]:
#showimp(rf, X, y)

In [8]:
#MAPPING
# rent_nonnum = rent_clean[non_num] /*/ numfeatures = ['bathrooms', 'bedrooms', 'longitude', 'latitude'] /*/ X = rent_clean[numfeatures + ['interest_level']] /*/ import category_encoders as ce encoder = ce.OrdinalEncoder(mapping = [{'col':'interest_level','mapping':{'low' : 1 , 'medium' :2, 'high':3}}]) /*/ encoder.fit(X) /*/ X = encoder.transform(X)
# non_num es una lista de variables non numeric

In [9]:
#rf, oob = evaluate(X, y) /*/ showimp(rf, X, y)

In [11]:
#ONE HOT
#X = rent_clean[numfeatures + ['manager_id']] /*/ encoder = ce.OneHotEncoder(cols=['manager_id']) /*/ encoder.fit(X) /*/ X = encoder.transform(X) /*/ rf, oob = evaluate(X, y)

In [12]:
#DUMMIE
#onehot = pd.get_dummies(df['Hydraulics_Flow'], prefix='Hydraulics_Flow', dtype=bool) /*/ onehot.head(3)

In [14]:
#NORMALIZE
# from pandas.api.types import is_object_dtype,is_string_dtype /*/ def df_normalize_string(df): /*/     for col in df.columns: /*/         if is_object_dtype(df[col]) or is_string_dtype(df[col]): /*/             df[col] = df[col].str.lower() /*/             df[col] = df[col].fillna(np.nan) /*/             df[col] = df[col].replace('none or unspecified', np.nan) /*/             df[col] = df[col].replace('none', np.nan) /*/             df[col] = df[col].replace('#name?', np.nan) /*/             df[col] = df[col].replace('', np.nan) 
#df['fiModelSeries'].unique()

In [19]:
#missing num
#def fix_missing_num(df,colname): /*/     df[colname+'_na'] = pd.isnull(df[colname]) /*/     df[colname].fillna(df[colname].median(), inplace=True)

In [18]:
#split dates
#def df_split_dates(df,colname): /*/     df["saleyear"] = df[colname].dt.year /*/     df["salemonth"] = df[colname].dt.month /*/     df["saleday"] = df[colname].dt.day /*/     df["saledayofweek"] = df[colname].dt.dayofweek /*/     df["saledayofyear"] = df[colname].dt.dayofyear/*/     df[colname] = df[colname].astype(np.int64)

In [24]:
## has apartment been renovated?
#df['renov'] = df['description'].str.contains("renov")

#for w in ['doorman', 'parking', 'garage', 'laundry', 
#          'Elevator', 'fitness center', 'dishwasher']:
#    df[w] = df['features'].str.contains(w)
#df[['doorman', 'parking', 'garage', 'laundry']].head(5)

* One-Hot Encoding: This method converts each category value into a new column and assigns a binary value (0 or 1) to each category. One-hot encoding is suitable when the categories are not ordinal and do not have a natural order. Useful when categories are not ordinal and do not have a natural order.
* Label Encoding: In this technique, each category is assigned a unique integer number. It's suitable for ordinal data, where there is a clear ordering among the categories. Suitable for ordinal categorical variables where there is a clear ordering among the categories.
* Ordinal Encoding: Similar to label encoding, ordinal encoding assigns integer values to categories. However, it considers the order of the categories and assigns integers accordingly. Similar to label encoding but explicitly considers the order of categories.
* Mapping: Mapping involves directly replacing categorical values with numerical values based on a predefined mapping dictionary. This can be useful when the categorical variable has a natural order or when you want to manually assign numerical values to categories.
* Target Encoding:  In target encoding, each category is replaced with the mean of the target variable for that category. Useful for classification tasks where the target variable is categorical.

# Desition tree

In [35]:
import numpy as np
from numpy.random import default_rng
rnd_list = default_rng(110)
x = rnd_list.integers(low=0, high=100, size = (6))
y = rnd_list.integers(low=0, high=100, size = (6))
#mid point = 60

In [36]:
x

array([59, 93, 63, 51, 23, 71], dtype=int64)

In [37]:
y

array([70, 50,  4, 23, 23, 87], dtype=int64)

In [4]:
def split_data(x,y,split_pt):
    mask = x > split_pt
    anti_mask = x < split_pt
    x_true = x[mask]
    y_true = y[mask]
    x_false = x[anti_mask]
    y_false = y[anti_mask]
    return x_true, y_true, x_false , y_false

In [5]:
x_true, y_true, x_false , y_false =split_data(x,y, 60)

In [6]:
def MAE(y, y_true, y_false):
    y_true_hat = np.mean(y_true)
    y_false_hat = np.mean(y_false)
    y_hat = np.mean(y)
    
    old_mae = np.mean(np.absolute(y-y_hat))
    new_mae = len(y_true)/ len(y) * np.mean(np.absolute(y_true-y_true_hat)) + \
              len(y_false)/ len(y) * np.mean(np.absolute(y_false-y_false_hat)) 
    return old_mae, new_mae

In [7]:
MAE(y,y_true,y_false)

(26.166666666666668, 24.77777777777778)

In [8]:
x= [10,15,20,25]

In [9]:
def split (x):
    split_points=[]
    x_sorted = sorted(x)
    for i in range(len(x)-1):
        mid_point= (x_sorted[i]+x_sorted[i+1])/2
        split_points.append(mid_point)
    return split_points

In [10]:
split(x)

[12.5, 17.5, 22.5]

In [15]:
def get_best_split(data):
    x = data[:,0]
    y = data[:,1]
    split_pts = split(x) #1
    results = {}
    for point in split_pts:
        x_true, y_true, x_false , y_false = split_data(x,y, point) #2
        errors = MAE(y, y_true, y_false) #3
        results[point] = errors
    best_point = min(results,key=results.get)
    return results,best_point

In [20]:
list = rnd_list.integers(low=5, high=100, size = (5,5))
list

array([[80, 25,  5,  9, 80],
       [83,  7, 36, 54, 44],
       [37, 37, 24, 57, 60],
       [87, 52, 32, 18, 73],
       [55, 95, 36, 91, 83]], dtype=int64)

In [19]:
get_best_split(list)

({48.5: (27.76, 21.400000000000002),
  64.5: (27.76, 8.333333333333334),
  72.0: (27.76, 18.599999999999998),
  77.5: (27.76, 22.400000000000002)},
 64.5)

### Desition tree with sklearn

In [65]:
X=np.array([[59, 93, 63, 51, 23, 71], [10, 93, 63, 80, 23, 71], [15, 83, 93, 91, 25, 71], [19, 90, 60, 50, 20, 70], [19, 13, 13, 11, 13, 11], [9, 3, 6, 5, 3, 1]])

In [66]:
y=np.array([70, 50,  4, 23, 23, 55])

In [67]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y) 

In [71]:
regr = DecisionTreeRegressor(random_state =122)

In [72]:
regr.fit(X, y)

DecisionTreeRegressor(random_state=122)

# Random Forest

In [21]:
from numpy.random import default_rng
import numpy as np

In [31]:
rand = default_rng(110)
x = rand.integers(0,800,(10,6))
x

array([[479, 744, 508, 409, 184, 570],
       [560, 401,  35, 191, 190, 701],
       [708,  55, 414, 562, 737,  78],
       [784, 356, 297, 419, 681, 211],
       [785,  51, 680, 509, 364, 392],
       [754, 512, 367, 350, 202, 570],
       [222, 561,  43, 307, 691, 665],
       [449, 674, 758, 374, 670, 655],
       [ 46, 304, 143, 499, 293, 453],
       [ 48, 624, 396, 574, 138, 762]], dtype=int64)

In [32]:
y = rand.integers(0,800,(10,1))
y

array([[733],
       [260],
       [637],
       [175],
       [  0],
       [ 40],
       [633],
       [663],
       [ 24],
       [262]], dtype=int64)

In [33]:
#1 boorstraping and obb
def bs_sampling(data):
    bs = rand.choice(data,data.shape[0],replace=True)
    unique_bs = np.unique(bs, axis=0)
    oob=[]
    for row in data:
        if sum(np.isin(row,unique_bs))==0:
            oob.append(row) 
    return bs,np.array(oob)

In [35]:
bs_sampling(x)

(array([[754, 512, 367, 350, 202, 570],
        [785,  51, 680, 509, 364, 392],
        [784, 356, 297, 419, 681, 211],
        [784, 356, 297, 419, 681, 211],
        [708,  55, 414, 562, 737,  78],
        [754, 512, 367, 350, 202, 570],
        [754, 512, 367, 350, 202, 570],
        [ 46, 304, 143, 499, 293, 453],
        [754, 512, 367, 350, 202, 570],
        [708,  55, 414, 562, 737,  78]], dtype=int64),
 array([[560, 401,  35, 191, 190, 701],
        [222, 561,  43, 307, 691, 665],
        [449, 674, 758, 374, 670, 655],
        [ 48, 624, 396, 574, 138, 762]], dtype=int64))

In [37]:
# this is only for 1 tree
from sklearn.tree import DecisionTreeRegressor

def create_tree(data):
    X = data[:,0:-1]
    y = data[:,-1]
    dt = DecisionTreeRegressor()
    dt.fit(X,y)
    return dt

In [38]:
create_tree(x)

DecisionTreeRegressor()

In [39]:
# proporcion de obb
oob_samples =[]
for i in range(1,100):
    bs = rand.choice(x,x.shape[0],replace=True)
    n_unique_samples = np.unique(bs, axis=0).shape[0]
    oob = (data.shape[0] - n_unique_samples) / x.shape[0]
    oob_samples.append(oob)
np.mean(oob_samples)

0.3505050505050505

## Case 2

In [200]:
#split data (not neccesary if we have who is x and whi is y) in case ir it neccesary do next)
rand = default_rng(42)
data = rand.integers(0,800,(10,6))
data

array([[ 71, 619, 523, 351, 346, 686],
       [ 68, 557, 161,  75, 421, 780],
       [588, 608, 573, 628, 410, 102],
       [671, 360, 400, 296, 146, 741],
       [625, 515, 321, 658, 436, 354],
       [360, 181,  73, 443, 710,  51],
       [686, 662, 221, 505, 132, 606],
       [560, 283,  54, 776, 356, 714],
       [542, 622, 607, 155, 291, 373],
       [398,  35, 437, 123, 594, 546]], dtype=int64)

In [204]:
x_a = np.array([[ 71, 619, 523, 351, 346, 686],
       [ 68, 557, 161,  75, 421, 780],
       [588, 608, 573, 628, 410, 102],
       [671, 360, 400, 296, 146, 741],
       [625, 515, 321, 658, 436, 354],
       [360, 181,  73, 443, 710,  51],
       [686, 662, 221, 505, 132, 606],
       [560, 283,  54, 776, 356, 714],
       [542, 622, 607, 155, 291, 373],
       [398,  35, 437, 123, 594, 546]])
x_a

array([[ 71, 619, 523, 351, 346, 686],
       [ 68, 557, 161,  75, 421, 780],
       [588, 608, 573, 628, 410, 102],
       [671, 360, 400, 296, 146, 741],
       [625, 515, 321, 658, 436, 354],
       [360, 181,  73, 443, 710,  51],
       [686, 662, 221, 505, 132, 606],
       [560, 283,  54, 776, 356, 714],
       [542, 622, 607, 155, 291, 373],
       [398,  35, 437, 123, 594, 546]])

In [205]:
x = x_a[:,:5]
y = x_a[:,-1]

In [208]:
y

array([686, 780, 102, 741, 354,  51, 606, 714, 373, 546])

In [48]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, random_state=0)

In [51]:
#CREATE MODEL
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(oob_score=True)
rf.fit(X_train, y_train)

RandomForestRegressor(oob_score=True)

In [52]:
#  Evaluating OOB after 10 runs
oob_score=[]
for i in range(10):
    rf= RandomForestRegressor(n_estimators=100, n_jobs= -1, oob_score= True)
    rf= rf.fit(X_train, y_train)
    oob_score.append(rf.oob_score_)

In [53]:
avg_oob_score = np.mean(oob_score)
avg_oob_score

-0.530413265648385

## Random Forest 3

In [54]:
x = data[:,:5]
y = data[:,-1]

In [57]:
# training model without split data
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 10)

rf.fit(x,y)

RandomForestRegressor(n_estimators=10)

In [60]:
#predict a random list of X values
import numpy as np
list = np.array([2,2,40,-73,5])
rf.predict([list])

array([599.2])

# Does the model capture training data relationships?

In [61]:
from sklearn.metrics import mean_absolute_error

In [63]:
# Avergae error without split data
preds = rf.predict(x)
e = mean_absolute_error(y,preds)
ep = e / y.mean() * 100

print(f"${e:.0f} average error; {ep:.2f}% error")

$106 average error; 21.34% error


In [66]:
#split data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train , y_test = train_test_split(x,y, test_size = 0.25)

In [67]:
rf = RandomForestRegressor(n_estimators = 10)

rf.fit(X_train,y_train)

RandomForestRegressor(n_estimators=10)

In [69]:
#average error after splitting data
preds = rf.predict(X_test)
e = mean_absolute_error(y_test,preds)
ep = e / y.mean() * 100

print(f"${e:.0f} average error; {ep:.2f}% error")

$388 average error; 78.29% error


### Hyper-parameters

In [71]:
rf_100 = RandomForestRegressor(n_estimators = 100)

rf_100.fit(X_train,y_train)

RandomForestRegressor()

In [72]:
preds = rf_100.predict(X_test)
e = mean_absolute_error(y_test,preds)
ep = e / y.mean() * 100

print(f"${e:.0f} average error; {ep:.2f}% error")

$390 average error; 78.82% error


## Feature importance

In [22]:
import pandas as pd
rent = pd.read_csv('rent.csv')

In [7]:
rent_clean = rent[(rent['price'] > 1000) & (rent['price'] < 10000)]
rent_clean = rent_clean[(rent_clean['longitude'] !=0) | (rent_clean['latitude']!=0)]
rent_clean = rent_clean[(rent_clean['latitude']>40.55) &
                        (rent_clean['latitude']<40.94) &
                        (rent_clean['longitude']>-74.1) &
                        (rent_clean['longitude']<-73.67)]

In [10]:
#define X and Y
numfeatures = ['bathrooms', 'bedrooms', 'longitude', 'latitude']
X = rent_clean[numfeatures]
y = rent_clean['price']

#create baseline model
rf = RandomForestRegressor(n_estimators=100, n_jobs=-1, oob_score=True)
rf.fit(X, y)
# get OOB from baseline model
oob_baseline = rf.oob_score_
print(f"Out-of-bag R^2 for baseline model is: {oob_baseline}")

Out-of-bag R^2 for baseline model is: 0.8680622153385407


In [27]:
# Create baseline model as a function showing rfnodes and maxdepth
def evaluate(X, y):
    rf = RandomForestRegressor(n_estimators=100, n_jobs=-1, oob_score=True)
    rf.fit(X, y)
    oob = rf.oob_score_
    n = rfnnodes(rf)
    h = np.median(rfmaxdepths(rf))
    print(f"OOB R^2 is {oob:.5f} using {n:,d} tree nodes with {h} median tree depth")
    return rf, oob

In [13]:
evaluate(X, y)

OOB R^2 is 0.86767 using 2,432,328 tree nodes with 35.0 median tree depth


(RandomForestRegressor(n_jobs=-1, oob_score=True), 0.8676653764742874)

In [16]:
# i have not done the split
features_ranking = importances(rf, X, y)
features_ranking

,Importance
Feature,
longitude,0.639354
bedrooms,0.614460
latitude,0.531310
bathrooms,0.476974


In [18]:
#after spliting
from sklearn.model_selection import train_test_split
X_train, X_test, y_train , y_test = train_test_split(X,y, test_size = 0.25)

In [19]:
rf = RandomForestRegressor(n_estimators = 10)
rf.fit(X_train,y_train)

RandomForestRegressor(n_estimators=10)

### Feature importance with RFPIMP

In [21]:
from rfpimp import importances

features_ranking = importances(rf, X_test, y_test)
print(features_ranking)

           Importance
Feature              
bedrooms     0.541811
longitude    0.530322
latitude     0.438200
bathrooms    0.428846


### Feature importance with Permutation Importance SCRATCH

In [23]:
def perm_importances(X, y):
    rf = RandomForestRegressor(n_estimators=100, n_jobs=-1, oob_score=True, random_state=999)
    rf.fit(X, y)
    r2 = rf.oob_score_
    print(f"Baseline R^2 with no columns permuted: {r2:.5f}\n")
    for col in X.columns:
        X_col = X.copy()
        X_col[col] = X_col[col].sample(frac=1).values
        rf.fit(X_col, y)
        r2_col = rf.oob_score_
        print(f"Permuting column {col}: new R^2 is {r2_col:.5f} and difference from baseline is {r2 - r2_col:.5f}")

In [24]:
perm_importances(X, y)

Baseline R^2 with no columns permuted: 0.86763

Permuting column bathrooms: new R^2 is 0.81411 and difference from baseline is 0.05352
Permuting column bedrooms: new R^2 is 0.75463 and difference from baseline is 0.11299
Permuting column longitude: new R^2 is 0.68235 and difference from baseline is 0.18528
Permuting column latitude: new R^2 is 0.68480 and difference from baseline is 0.18283


### Feature importance with Dropped Column Importance SCRATCH

In [25]:
def drop_importances(X, y):
    rf = RandomForestRegressor(n_estimators=100, n_jobs=-1, oob_score=True, random_state=999)
    rf.fit(X, y)
    r2 = rf.oob_score_
    print(f"Baseline R^2 with no columns dropped: {r2:.5f}\n")
    for col in X.columns:
        X_col = X.copy()
        X_col = X_col.drop(col, axis=1) 
        rf.fit(X_col, y)
        r2_col = rf.oob_score_
        print(f"Dropping column {col}: new R^2 is {r2_col:.5f} and difference from baseline is {r2 - r2_col:.5f}")

In [26]:
drop_importances(X, y)

Baseline R^2 with no columns dropped: 0.86763

Dropping column bathrooms: new R^2 is 0.81941 and difference from baseline is 0.04821
Dropping column bedrooms: new R^2 is 0.77719 and difference from baseline is 0.09044
Dropping column longitude: new R^2 is 0.73451 and difference from baseline is 0.13311
Dropping column latitude: new R^2 is 0.74144 and difference from baseline is 0.12619


In [107]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import io
import requests
from sklearn.ensemble import RandomForestClassifier

In [114]:
# EJERCICIO feature importance 
#If data1.sample(n=91,random_state=54) is your data where 'sex' is target, what would be the feature importance for feature 'hip.gi'
# with using baseline random forest model with 77 trees and random_state = 54?
df_url = 'https://raw.githubusercontent.com/akmand/datasets/master/openintro/bdims.csv'
url_content = requests.get(df_url, verify=False).content
data1 = pd.read_csv(io.StringIO(url_content.decode('utf-8')))

In [109]:
df= data1.sample(n=91, random_state=54)

In [110]:
clf = RandomForestClassifier(n_estimators=77,random_state=54)

In [111]:
x= df.drop('sex', axis=1)
y= df[['sex']]

In [112]:
clf.fit(x,y)

RandomForestClassifier(n_estimators=77, random_state=54)

In [113]:
pd.Series(index = cl.feature_names_in_, data = cl.feature_importances_)

radius error            0.300708
texture error           0.081460
concave points error    0.140882
symmetry error          0.073412
worst texture           0.176907
worst smoothness        0.103266
worst symmetry          0.123364
dtype: float64

# Clasification RandomForest

In [29]:
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer()
X = cancer.data
y = cancer.target

df = pd.DataFrame(X, columns = cancer.feature_names)

In [30]:
features = ['radius error','texture error','concave points error','symmetry error',
          'worst texture','worst smoothness','worst symmetry']

df= df[features]

In [31]:
X_train, X_test, y_train , y_test = train_test_split(df,y, test_size = 0.20)
cl = RandomForestClassifier(n_estimators=200)
cl.fit(X_train,y_train)

RandomForestClassifier(n_estimators=200)

In [33]:
from sklearn.metrics import accuracy_score
preds = cl.predict(X_test)
e = accuracy_score(y_test,preds)
print(f"{e*100:.2f}% accuracy")

93.86% accuracy


In [34]:
features_ranking = importances(cl, X_test, y_test)
features_ranking

,Importance
Feature,
radius error,0.192982
concave points error,0.096491
worst texture,0.052632
symmetry error,0.043860
texture error,0.026316
worst smoothness,0.008772
worst symmetry,0.008772


# KNN

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.02 , random_state = 110)

In [ ]:
def euclidean_distance(point1, point2):
    return np.sqrt(sum((point1-point2)**2))

#df['euclidean_distance'] = np.sqrt((df['Lattitude'] - df['x1'])**2 + (df['Longtitude'] - df['y2'])**2)
#df.iloc[[500]]

In [ ]:
def manhattan_distance(point1, point2):
    return np.sum(np.abs(point1-point2))

In [ ]:
def minkowski_distance(point1, point2,p):
    return np.power(sum(np.abs(point1-point2)**p),1/p)

In [ ]:
def predict_price(X_train,y_train,sample,k=5,p):
    distances = []
    for x in X_train.to_numpy():
        distance = minkowski_distance(sample,x,p)
        distances.append(distance)
    sorted_distances = np.argsort(distances)
    k_nearest_targets = y_train[sorted_distances[:k]]
    predict_price = np.mean(k_nearest_targets)
    return predict_price

In [ ]:
def predict_class(X_train,y_train,sample,k=5,p):
    distances = []
    for x in X_train.to_numpy():
        distance = minkowski_distance(sample,x,p)
        distances.append(distance)
    sorted_distances = np.argsort(distances)
    k_nearest_targets = y_train[sorted_distances[:k]]
    class_counts = np.bincount(k_nearest_targets)
    prediction = agrmax(class_counts)
    return prediction

In [ ]:
def calculate_mse_for_k(X_train,y_train,X_test,y_test,k,p):
    y_preds= []
    for sample in X_test.to_numpy():
        y_pred = predict_price(X_train,y_train,sample,k,p)
        y_preds.append(y_pred)
    mse = mean_squared_error(y_test,y_preds)
    return mse

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.02 , random_state = 110)
knn_regressor = KNeighborsRegressor(n_neighbors=11,p=3)
knn_regressor.fit(X_train,y_train)
y_pred = knn_regressor.predict(X_test)
mse = mean_squared_error(y_test,y_pred)
mse

Given the true values y = np.array and the predictions from your model preds, what is the mean squared error?

In [73]:
y = np.array([0.69882258, 0.41020411, 0.97465025, 0.36357668, 0.92831302, 0.98208047, 0.43229378, 0.21032851, 0.80564111, 0.9221681, 0.34010773, 0.51208259, 0.30784147, 0.50252704, 0.43137395, 0.69055875, 0.96634857, 0.20403823, 0.05456629, 0.25382772, 0.37175274, 0.7149425, 0.22146558, 0.02238627, 0.40732219])

In [75]:
preds = np.array([0.64879474, 0.39564339, 0.79922137, 0.18259129, 0.75875667, 0.98423633, 0.02784706, 0.71532278, 0.63592899, 0.92868504, 0.74281343, 0.01484505, 0.39772027, 0.41268879, 0.01877527, 0.25117644, 0.45016074, 0.606493, 0.69795758, 0.52579034, 0.93864457, 0.58332547, 0.06342663, 0.85158433, 0.64288857])

In [80]:
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y,preds)
mse

0.1341745546868982

With x, y, and your split point = 14.5, what is the value of the mean absolute error after the split?

In [99]:
X = np.array([ 7, 13, 16, 1, 41, 2])

In [100]:
y = np.array([7, 7, 7, 6, 7, 4])

In [101]:
split_pt = 14.5

In [98]:
def split_data(x,y,split_pt):
    mask = x > split_pt
    anti_mask = x < split_pt
    x_true = x[mask]
    y_true = y[mask]
    x_false = x[anti_mask]
    y_false = y[anti_mask]
    return x_true, y_true, x_false , y_false

In [102]:
x_true, y_true, x_false , y_false =split_data(X,y, 14.5)

In [103]:
def MAE(y, y_true, y_false):
    y_true_hat = np.mean(y_true)
    y_false_hat = np.mean(y_false)
    y_hat = np.mean(y)
    
    old_mae = np.mean(np.absolute(y-y_hat))
    new_mae = len(y_true)/ len(y) * np.mean(np.absolute(y_true-y_true_hat)) + \
              len(y_false)/ len(y) * np.mean(np.absolute(y_false-y_false_hat)) 
    return old_mae, new_mae

In [104]:
MAE(y,y_true,y_false)

(0.888888888888889, 0.6666666666666666)